# FHIR for Research Workshop - Exercise 3

## Drug on Drug Interactions

 For this exercise we will explore potential drug on drug interactions in a patient cohort by drawing from the NIH's Drug RxNAV database that includes open APIs we can reference. 

For this exercise we will explore the following scenario.


## Scenario: Potential Drug on Drug Interaction Risks in a Patient Population
For this scenario we will pull the list of prescriptions from our patient cohort, and then leveraging the NIH Drug on Drug API to determine if there is any potential risk of adverse events to patients. 

For this initial analysis we will want to do the following:
### Key Activities:
<ol>
    <li> Query all active prescriptions in our patient cohort</li>
    <li> Determine how to query the API and then construct a mechanism to feed our patient info into that API to determine if a known Drug on Drug Interaction could occur.</li>
    <li> Construct a composite list of all drugs per-patient (so we can determine a potential Drug on Drug interaction</li>
    <li> Construct a program to loop through our entire cohort and determine the aggregate risk.</li>

### Motivation/Purpose
From a research persective we can envision leveraging these sorts of analyses to do post-market surveilance of drugs to determine both the rate of known adverse events among patients, as well as to potentially flag additional risks not yet identified. 

Clinically this exercise demonstrates the power of a SMART of FHIR application, where third-party data (in this case Drug on Drug interaction data), can be pulled in, paired with FHIR formatted clinical data, and then leveraged to better inform patient care in the form of Clinical Decision Support tools. 

### Key Skills Practiced
For this scenario, the key challenge will be in mapping the API query to our data and constructing a data structure capable of leveraging it automatically. 

## Step 1 Query all active prescriptions in our patient cohort

For this exercise we will call on the 'MedicationRequest' resource which is the closest equivalent to a prescriptions resource in FHIR. Each item is effectively a single prescription. 

We will also make sure to include the relevant patient information to ensure we can map multiple prescritpions to individual patients.

In [1]:
import requests
import json
import pandas as pd
from pandas import json_normalize

In [2]:
r = requests.get(f"https://api.logicahealth.org/researchonfhir/open/MedicationRequest?", verify=False)
bundle = r.json()

/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [3]:
bundle

{'resourceType': 'Bundle',
 'id': '69ed0eac-e119-4d89-9ffb-da596ccaf5ac',
 'meta': {'lastUpdated': '2022-01-12T08:03:44.154+00:00'},
 'type': 'searchset',
 'link': [{'relation': 'self',
   'url': 'https://api.logicahealth.org/researchonfhir/open/MedicationRequest'},
  {'relation': 'next',
   'url': 'https://api.logicahealth.org/researchonfhir/open?_getpages=69ed0eac-e119-4d89-9ffb-da596ccaf5ac&_getpagesoffset=50&_count=50&_pretty=true&_bundletype=searchset'}],
 'entry': [{'fullUrl': 'https://api.logicahealth.org/researchonfhir/open/MedicationRequest/smart-MedicationRequest-101',
   'resource': {'resourceType': 'MedicationRequest',
    'id': 'smart-MedicationRequest-101',
    'meta': {'versionId': '1',
     'lastUpdated': '2020-07-15T02:51:25.000+00:00',
     'source': '#KQSArAdbxORTtqVw'},
    'text': {'status': 'generated',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml">Nizatidine 15 MG/ML Oral Solution [Axid] (rxnorm: 582620)</div>'},
    'status': 'active',
    'intent': 'or

Let's borrow the list mapping lambda function we deployed to map out our JSON file and then return the first example so we can get a better look at what information we have to work with.

In [4]:
prescriptions = list(map(lambda e: e['resource'], bundle['entry']))
prescriptions[0]

{'resourceType': 'MedicationRequest',
 'id': 'smart-MedicationRequest-101',
 'meta': {'versionId': '1',
  'lastUpdated': '2020-07-15T02:51:25.000+00:00',
  'source': '#KQSArAdbxORTtqVw'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml">Nizatidine 15 MG/ML Oral Solution [Axid] (rxnorm: 582620)</div>'},
 'status': 'active',
 'intent': 'order',
 'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
    'code': '582620',
    'display': 'Nizatidine 15 MG/ML Oral Solution [Axid]'}],
  'text': 'Nizatidine 15 MG/ML Oral Solution [Axid]'},
 'subject': {'reference': 'Patient/smart-1032702'},
 'dosageInstruction': [{'text': '10 mL bid',
   'timing': {'repeat': {'boundsPeriod': {'start': '2008-04-05'},
     'frequency': 2,
     'period': 1,
     'periodUnit': 'd'}}}],
 'dispenseRequest': {'numberOfRepeatsAllowed': 1,
  'quantity': {'value': 1.0,
   'unit': 'mL',
   'system': 'http://unitsofmeasure.org',
   'code': 'm

### Mount Data onto Pandas Dataframe

Now that we've extracted information we need, we will then take the FHIR formatted data and convert it into a pandas dataframe for subsequent analysis.

Based on our previous exercises we know we can use the JSON_normalize function parse the JSON into a pandas dataframe. Let's do that now and take a look at what we have

In [5]:
dfprescriptions = pd.json_normalize(prescriptions)
dfprescriptions.head()

,resourceType,id,status,intent,dosageInstruction,meta.versionId,meta.lastUpdated,meta.source,text.status,text.div,...,subject.reference,dispenseRequest.numberOfRepeatsAllowed,dispenseRequest.quantity.value,dispenseRequest.quantity.unit,dispenseRequest.quantity.system,dispenseRequest.quantity.code,dispenseRequest.expectedSupplyDuration.value,dispenseRequest.expectedSupplyDuration.unit,dispenseRequest.expectedSupplyDuration.system,dispenseRequest.expectedSupplyDuration.code
0,MedicationRequest,smart-MedicationRequest-101,active,order,"[{'text': '10 mL bid', 'timing': {'repeat': {'...",1,2020-07-15T02:51:25.000+00:00,#KQSArAdbxORTtqVw,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Niza...",...,Patient/smart-1032702,1,1.0,mL,http://unitsofmeasure.org,mL,30,days,http://unitsofmeasure.org,d
1,MedicationRequest,smart-MedicationRequest-102,active,order,"[{'text': '7 mL bid x 10 days', 'timing': {'re...",1,2020-07-15T02:51:26.000+00:00,#WnCTEkK79sEBIQNe,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Amox...",...,Patient/smart-1081332,1,1.0,mL,http://unitsofmeasure.org,mL,10,days,http://unitsofmeasure.org,d
2,MedicationRequest,smart-MedicationRequest-103,active,order,"[{'text': '5 mL bid x 10 days', 'timing': {'re...",1,2020-07-15T02:51:26.000+00:00,#WnCTEkK79sEBIQNe,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Amox...",...,Patient/smart-1081332,1,1.0,mL,http://unitsofmeasure.org,mL,10,days,http://unitsofmeasure.org,d
3,MedicationRequest,smart-MedicationRequest-104,active,order,"[{'text': '15 mL daily x 3 days', 'timing': {'...",1,2020-07-15T02:51:26.000+00:00,#WnCTEkK79sEBIQNe,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Azit...",...,Patient/smart-1081332,1,1.0,mL,http://unitsofmeasure.org,mL,3,days,http://unitsofmeasure.org,d
4,MedicationRequest,smart-MedicationRequest-105,active,order,"[{'text': '7 mL bid x 10 days', 'timing': {'re...",1,2020-07-15T02:51:26.000+00:00,#WnCTEkK79sEBIQNe,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">cefd...",...,Patient/smart-1081332,1,1.0,mL,http://unitsofmeasure.org,mL,10,days,http://unitsofmeasure.org,d


As we can see, certain fields, such as status and even subject.reference are immediately usable in their current form. For others, specifically the actual RX code, we're going to need to do further work. Let's go back to our original mapped JSON file and try to do a list comprehension to extract the specific code.

Then wrapping that within a Series function and executing a to_frame() method on the resulting series, we can create a dataframe with our desired RXCode

In [6]:
rxcodes = pd.Series([codings['code'] for MedicationRequest in prescriptions for codings in MedicationRequest['medicationCodeableConcept']['coding']], name='rxcode')

dfcode = rxcodes.to_frame()

In [7]:
dfcode.head()

,rxcode
0,582620
1,308189
2,617993
3,211307
4,261091


So we now have a basic datafame with drug and patient information. Before we can begin trying to construct a parser, we need to examine our API to see how data is submitted and returned.

## Step 2: Determine how to query the API and then construct a mechanism to feed our patient info into that API to determine if a known Drug on Drug Interaction could occur.

Reviewing the NIH's RX Norm database documentation. Link here: https://lhncbc.nlm.nih.gov/RxNav/APIs/index.html

We see one clear option we have to use is the RX CUI code using the six-digit NDC code
https://lhncbc.nlm.nih.gov/RxNav/APIs/api-RxNorm.getNDCs.html

This correllates with our Patient data column: resource.medicationCodeableConcept.codingcodes (quite a mouthful!, but we'll deal with that shortly).

Let's pull a sample interaction using the following general notation:

URL/list.json?rxcuis=[code 1]+[code 2]

In [8]:
url = 'https://rxnav.nlm.nih.gov/REST/interaction/list.json?rxcuis=207106+656659'
response = (requests.get(url).text)
response_json = json.loads(response)
response_json

{'nlmDisclaimer': 'It is not the intention of NLM to provide specific medical advice, but rather to provide users with information to better understand their health and their medications. NLM urges you to consult with a qualified physician for advice about medications.',
 'fullInteractionTypeGroup': [{'sourceDisclaimer': 'DrugBank is intended for educational and scientific research purposes only and you expressly acknowledge and agree that use of DrugBank is at your sole risk. The accuracy of DrugBank information is not guaranteed and reliance on DrugBank shall be at your sole risk. DrugBank is not intended as a substitute for professional medical advice, diagnosis or treatment..[www.drugbank.ca]',
   'sourceName': 'DrugBank',
   'fullInteractionType': [{'comment': 'Drug1 (rxcui = 207106, name = fluconazole 50 MG Oral Tablet [Diflucan], tty = SBD). Drug2 (rxcui = 656659, name = bosentan 125 MG Oral Tablet, tty = SCD). Drug1 is resolved to fluconazole, Drug2 is resolved to bosentan and 

We now have a target to work toward! For each patient, we need to compile a list of RXCUI codes and then append them to our NIH API query with a '+' between each code! Let's co about constructing that!

## Step 3: Construct a composite list of all drugs per-patient (so we can determine a potential Drug on Drug interaction

As a first step let's extract the three features we need: 'subject.reference', 'status' and 'rxcode'!

Let's now clean up our dataframe, by purging the columns we don't intend to use, and adding our code columns

In [9]:
dffinal = dfprescriptions[['subject.reference', 'status']]
dffinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   subject.reference  50 non-null     object
 1   status             50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes


In [10]:
dffinal['rxcode'] = dfcode['rxcode']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
dffinal.head()

,subject.reference,status,rxcode
0,Patient/smart-1032702,active,582620
1,Patient/smart-1081332,active,308189
2,Patient/smart-1081332,active,617993
3,Patient/smart-1081332,active,211307
4,Patient/smart-1081332,active,261091


Now we are ready to rumble! Let's now use a modified GroupBy function to merge our drugs by patient.

In [12]:
groups_by_patient = dffinal.groupby('subject.reference', sort=False)['rxcode'].apply(lambda x: x.values.tolist())
groups_by_patient.head()

subject.reference
Patient/smart-1032702                                             [582620]
Patient/smart-1081332    [308189, 617993, 211307, 261091, 404630, 23919...
Patient/smart-1098667             [150840, 309462, 310333, 351396, 310893]
Patient/smart-1134281    [239191, 352027, 104884, 755272, 308189, 10634...
Patient/smart-1137192    [308607, 312961, 381056, 866511, 866514, 19758...
Name: rxcode, dtype: object

ok so now we've generated a list of active prescriptions for each patient, we can now append this list to the RXNav query and determine whether each of these patients may have a drug on drug interaction

## Step 4: Construct a program to loop through our entire cohort and determine the aggregate risk.

Ok so to recap: we now have a list of patients with associated rx codes in list form, and we have a mechanism to query the RXNav API to determin if a Drug on Drug interaction exists. As a last step let's create a series of functions to iterate through our patient list and for each patient return whether or not a Drug on Drug interaction could occur

In [13]:
# Declare variables for index counting and sum total of drug interactions
index = 1
count_drug_int = 0

In [14]:
# Function for calling NIH API
def get_api_data(drug_list):
    try:
        url = 'https://rxnav.nlm.nih.gov/REST/interaction/list.json?rxcuis=' + drug_list
        response = (requests.get(url).text)
        response_json = json.loads(response)
        return response_json

    except Exception as e:
        raise e

In [15]:
# Iterate through each patient list of medications
for drug_list in groups_by_patient:
    joined_drug_list = "+".join(str(i) for i in drug_list)
    # print(i, " " , drug_list)
    index += 1
    data = get_api_data(joined_drug_list) # returns JSON response
    if 'fullInteractionTypeGroup' not in data:
        print('No drug interaction: ', joined_drug_list)
        continue
    count_drug_int += 1
    print('Drug interaction: ', joined_drug_list)

print('Total number of drug interactions: ', count_drug_int)

No drug interaction:  582620
Drug interaction:  308189+617993+211307+261091+404630+239191+208406
No drug interaction:  150840+309462+310333+351396+310893
Drug interaction:  239191+352027+104884+755272+308189+106346+617423
Drug interaction:  308607+312961+381056+866511+866514+197589+762675
No drug interaction:  151124+877300+311992+285128
Drug interaction:  745679+404473+284429+404630+352318+352319+745752
Drug interaction:  314077+404673+153357+197770+199381
No drug interaction:  106256+197803+313960+198305+309367
No drug interaction:  762675+859258
Total number of drug interactions:  5


## Summary

So this is a really interesting example of how FHIR data can interact with the broader ecosystem of Healthcare data and resources to determine additional health care insights. Here we pulled data from multiple resources into a unified dataframe, and then modified how the data was stored in order to pass it through to a third-party API and determine health outcomes.